In [1]:
import numpy as np
from optimizers import cmaes as opt
import objectives as obj
from optimizers._optimizer import _Optimizer
from optimizers.optimfactory import OptimizerFactory
from experiments.expt import _ColorPrint

# Common Settings

In [2]:
# experimental settings
max_ite = 10000     # maximum num. of iterations
dim = 100           # num. of total dimensions
eff_dim = 10        # num. of effective dimensions
seed = 1            # seed

###
# objective functions
# list: "Ackley", "Sphere", "Ellipsoid", "Rosenbrock", "AttractiveSector", "SharpRidge"
###

# obj_name = "Ackley"
obj_name = "Sphere"
# obj_name = "Ellipsoid"
# obj_name = "Rosenbrock"
# obj_name = "AttractiveSector"
# obj_name = "SharpRidge"

In [3]:
def run_experiment(objective_function, optimizer):
    best = np.inf
    func_info = f"{objective_function.__class__.__name__} ({objective_function.dimensionality}, {objective_function.effective_dimensionality})"
    
    print(
        "\r",
        f"#(iterations, evaluations) BestEval, FunctionInfo",
    )

    for iter_i in range(max_ite):
        solutions = list()
        prev_best = best
        for _ in range(optimizer.pop_size):
            x = optimizer.ask()
            value = objective_function(x)
            try:
                sample_idx = optimizer.sample_idx
            except:
                sample_idx = -1
            solutions.append((x, value, sample_idx))
            
            if best > value:
                    best = value
                    
                    if objective_function.is_optimized(x=x):
                        _ColorPrint.green(
                            f"\n[Optimized] #({optimizer.generations}, {optimizer.num_of_feval}) {np.round(best, 10)}, {func_info}"
                        )
                        return optimizer.num_of_feval, best
                
        if (prev_best > best) or (iter_i < 10) or (iter_i % 100 == 0):
            
            print(
                "\r",
                f"#({optimizer.generations}, {optimizer.num_of_feval}) {np.round(best, 10)}, {func_info}",
            )
        
        optimizer.tell(solutions=solutions)
        
    return optimizer.num_of_feval, best

In [4]:
# we left readable code
def run_simplified_experiment(objective_function, optimizer):
    best = np.inf

    for _ in range(max_ite):
        solutions = list()
        for _ in range(optimizer.pop_size):
            x = optimizer.ask()
            value = objective_function(x)
            try:
                sample_idx = optimizer.sample_idx
            except:
                sample_idx = -1
            solutions.append((x, value, sample_idx))
            
            if best > value:
                    best = value
                    if objective_function.is_optimized(x=x):
                        return optimizer.num_of_feval, best
        
        optimizer.tell(solutions=solutions)
        
    return optimizer.num_of_feval, best

## Run sep-CMA-ES (CSA)

In [5]:
method = "SepCMAES"

# set objective function
objective_function = obj.ContinuousObjectiveFunctionFactory.get(
        name=obj_name,
        dimensionality=dim,
        effective_dimensionality=eff_dim,
        terminate_condition=1e-8,
        seed=seed,
    )

# set optimizer
optimizer = OptimizerFactory.get(method=method, obj_name=obj_name, dim=dim, seed=seed, lam=0)

In [6]:
# run experiment
res = run_experiment(objective_function, optimizer)

# simplified experiment (no print on terminal)
# res = run_simplified_experiment(objective_function, optimizer)  

_ColorPrint.blue(f"\nNum. of evaluations: {res[0]}, Best evaluation value: {res[1]}")

 #(iterations, evaluations) BestEval, FunctionInfo
 #(0, 17) 97.1919842149, Sphere (100, 10)
 #(1, 34) 54.5440941122, Sphere (100, 10)
 #(2, 51) 54.2886704887, Sphere (100, 10)
 #(3, 68) 23.7164444925, Sphere (100, 10)
 #(4, 85) 7.9012117028, Sphere (100, 10)
 #(5, 102) 7.9012117028, Sphere (100, 10)
 #(6, 119) 7.9012117028, Sphere (100, 10)
 #(7, 136) 7.9012117028, Sphere (100, 10)
 #(8, 153) 7.9012117028, Sphere (100, 10)
 #(9, 170) 7.9012117028, Sphere (100, 10)
 #(12, 221) 7.3976253446, Sphere (100, 10)
 #(13, 238) 5.6963500987, Sphere (100, 10)
 #(23, 408) 5.4211750284, Sphere (100, 10)
 #(30, 527) 5.2791433581, Sphere (100, 10)
 #(41, 714) 2.8706737386, Sphere (100, 10)
 #(50, 867) 2.4688728556, Sphere (100, 10)
 #(51, 884) 2.4671105064, Sphere (100, 10)
 #(76, 1309) 2.36987561, Sphere (100, 10)
 #(82, 1411) 2.1311634171, Sphere (100, 10)
 #(86, 1479) 1.8636388527, Sphere (100, 10)
 #(88, 1513) 1.3595645484, Sphere (100, 10)
 #(95, 1632) 1.3592459814, Sphere (100, 10)
 #(100, 171

# Run sep-CMA-ES-LED (CSA)

In [7]:
method = "SepCMAESLED"
add_kwargs = {
    "gain_power_min": -1,
    "gain_power_max": 5,
    "beta_hat": 0.01
}

# set objective function
objective_function = obj.ContinuousObjectiveFunctionFactory.get(
        name=obj_name,
        dimensionality=dim,
        effective_dimensionality=eff_dim,
        terminate_condition=1e-8,
        seed=seed,
    )

# set optimizer
optimizer = OptimizerFactory.get(method=method, obj_name=obj_name, dim=dim, seed=seed, lam=0)
for k, v in add_kwargs.items():
    setattr(optimizer, k, v)

In [8]:
# run experiment
res = run_experiment(objective_function, optimizer)

# simplified experiment (no print on terminal)
# res = run_simplified_experiment(objective_function, optimizer)  

_ColorPrint.blue(f"\nNum. of evaluations: {res[0]}, Best evaluation value: {res[1]}")

 #(iterations, evaluations) BestEval, FunctionInfo
 #(0, 17) 97.1919842149, Sphere (100, 10)
 #(1, 34) 54.3226535636, Sphere (100, 10)
 #(2, 51) 53.9805090013, Sphere (100, 10)
 #(3, 68) 23.7511811599, Sphere (100, 10)
 #(4, 85) 8.635994899, Sphere (100, 10)
 #(5, 102) 8.635994899, Sphere (100, 10)
 #(6, 119) 8.635994899, Sphere (100, 10)
 #(7, 136) 8.635994899, Sphere (100, 10)
 #(8, 153) 8.635994899, Sphere (100, 10)
 #(9, 170) 8.635994899, Sphere (100, 10)
 #(13, 238) 8.3095035059, Sphere (100, 10)
 #(23, 408) 6.2048140322, Sphere (100, 10)
 #(31, 544) 5.373587409, Sphere (100, 10)
 #(41, 714) 3.5409170085, Sphere (100, 10)
 #(42, 731) 3.273727269, Sphere (100, 10)
 #(50, 867) 2.7812436331, Sphere (100, 10)
 #(76, 1309) 2.7800084707, Sphere (100, 10)
 #(81, 1394) 2.1229667415, Sphere (100, 10)
 #(82, 1411) 1.0264696433, Sphere (100, 10)
 #(86, 1479) 0.7287174249, Sphere (100, 10)
 #(88, 1513) 0.7031322478, Sphere (100, 10)
 #(92, 1581) 0.6158808411, Sphere (100, 10)
 #(93, 1598) 0.5

# Run sep-CMA-ES (TPA)

In [9]:
method = "SepCMAESModifiedTPA"

# set objective function
objective_function = obj.ContinuousObjectiveFunctionFactory.get(
        name=obj_name,
        dimensionality=dim,
        effective_dimensionality=eff_dim,
        terminate_condition=1e-8,
        seed=seed,
    )

# set optimizer
optimizer = OptimizerFactory.get(method=method, obj_name=obj_name, dim=dim, seed=seed, lam=0)

In [10]:
# run experiment
res = run_experiment(objective_function, optimizer)

# simplified experiment (no print on terminal)
# res = run_simplified_experiment(objective_function, optimizer)  

_ColorPrint.blue(f"\nNum. of evaluations: {res[0]}, Best evaluation value: {res[1]}")

 #(iterations, evaluations) BestEval, FunctionInfo
 #(0, 17) 97.1919842149, Sphere (100, 10)
 #(1, 34) 54.2909299319, Sphere (100, 10)
 #(2, 51) 54.2909299319, Sphere (100, 10)
 #(3, 68) 38.1695505397, Sphere (100, 10)
 #(4, 85) 22.3128798841, Sphere (100, 10)
 #(5, 102) 22.3128798841, Sphere (100, 10)
 #(6, 119) 21.7827814784, Sphere (100, 10)
 #(7, 136) 21.7827814784, Sphere (100, 10)
 #(8, 153) 18.8914368564, Sphere (100, 10)
 #(9, 170) 18.8914368564, Sphere (100, 10)
 #(10, 187) 10.8053322284, Sphere (100, 10)
 #(13, 238) 5.6809293616, Sphere (100, 10)
 #(23, 408) 4.2936853769, Sphere (100, 10)
 #(32, 561) 3.6903993323, Sphere (100, 10)
 #(35, 612) 1.6759651633, Sphere (100, 10)
 #(40, 697) 1.4885149602, Sphere (100, 10)
 #(41, 714) 1.4000109347, Sphere (100, 10)
 #(42, 731) 0.9364415362, Sphere (100, 10)
 #(45, 782) 0.7210546672, Sphere (100, 10)
 #(48, 833) 0.4939898282, Sphere (100, 10)
 #(49, 850) 0.3035491739, Sphere (100, 10)
 #(51, 884) 0.2409476593, Sphere (100, 10)
 #(58, 

# Run sep-CMA-ES-LED (TPA)

In [11]:
method = "SepCMAESModifiedTPALED"

# set objective function
objective_function = obj.ContinuousObjectiveFunctionFactory.get(
        name=obj_name,
        dimensionality=dim,
        effective_dimensionality=eff_dim,
        terminate_condition=1e-8,
        seed=seed,
    )

# set optimizer
optimizer = OptimizerFactory.get(method=method, obj_name=obj_name, dim=dim, seed=seed, lam=0)
for k, v in add_kwargs.items():
    setattr(optimizer, k, v)

In [12]:
# run experiment
res = run_experiment(objective_function, optimizer)

# simplified experiment (no print on terminal)
# res = run_simplified_experiment(objective_function, optimizer)  

_ColorPrint.blue(f"\nNum. of evaluations: {res[0]}, Best evaluation value: {res[1]}")

 #(iterations, evaluations) BestEval, FunctionInfo
 #(0, 17) 97.1919842149, Sphere (100, 10)
 #(1, 34) 54.2869933866, Sphere (100, 10)
 #(2, 51) 54.2869933866, Sphere (100, 10)
 #(3, 68) 38.241210622, Sphere (100, 10)
 #(4, 85) 22.4920839031, Sphere (100, 10)
 #(5, 102) 22.4920839031, Sphere (100, 10)
 #(6, 119) 21.4832001716, Sphere (100, 10)
 #(7, 136) 21.4832001716, Sphere (100, 10)
 #(8, 153) 19.8709813803, Sphere (100, 10)
 #(9, 170) 19.8709813803, Sphere (100, 10)
 #(10, 187) 9.9199407997, Sphere (100, 10)
 #(13, 238) 5.6918203598, Sphere (100, 10)
 #(23, 408) 4.461416571, Sphere (100, 10)
 #(29, 510) 4.2488980469, Sphere (100, 10)
 #(30, 527) 4.2366336276, Sphere (100, 10)
 #(32, 561) 2.7087350729, Sphere (100, 10)
 #(35, 612) 2.1861030235, Sphere (100, 10)
 #(37, 646) 1.8625900018, Sphere (100, 10)
 #(38, 663) 1.3780575016, Sphere (100, 10)
 #(40, 697) 1.0841533763, Sphere (100, 10)
 #(41, 714) 0.9954828556, Sphere (100, 10)
 #(42, 731) 0.5719623987, Sphere (100, 10)
 #(46, 799